# Machine Learning
# Exercise 3 - Kaggle competition
<br/>Student:
<br/>se21m024
<br/>Matriculation number: 1425616
<br/>Thomas Stummer
<br/><br/>The interpretation of the data can be found in the document <b><i>se21m024_Stummer_ml_ex3_breastcancer.pdf</i></b>.
<br/><br/>
Small data set: Breast Cancer

## Import Dependencies

In [18]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import datetime
import pandas as pd
import glob, os
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import datetime
from collections import deque
from sklearn.model_selection import cross_val_score
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
import numpy as np
import scipy.stats.stats as st
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import uuid

# Matriculation number: 1425616
random_state = 1425616
use_kaggle_paths = False

## Import Data

In [19]:
data_set_name = 'breastcancer'

# Training data
train_data_set_file = ".\\data\\breast-cancer-diagnostic.shuf.lrn.csv" # Local dev
if use_kaggle_paths:
    train_data_set_file = "../input/mse-bb-2-ss2022-mle-breastcancer/breast-cancer-diagnostic.shuf.lrn.csv"  # Kaggle

train_data_set = pd.read_csv(train_data_set_file)

# Split data in input features (X) and target (y) feature
train_data_set_X = train_data_set.loc[:,'radiusMean':]
train_data_set_y = train_data_set.loc[:,'class']

# Testing data
test_data_set_file = ".\\data\\breast-cancer-diagnostic.shuf.tes.csv" # Local dev
if use_kaggle_paths:
    test_data_set_file = "../input/mse-bb-2-ss2022-mle-breastcancer/breast-cancer-diagnostic.shuf.tes.csv"  # Kaggle
    
test_data_set = pd.read_csv(test_data_set_file)
test_data_set_X = test_data_set.loc[:,'radiusMean':]

## Classifiers

In [20]:
# k-NN
# kd tree was chosen to gain results within a reasonable amount of time
def kNN (dataSetName, X_train, X_test, y_train, y_test, n_neighbors_values):

    results = []
    bestResult = None

    for n_neighbors in n_neighbors_values:
            classifier = KNeighborsClassifier(n_neighbors=n_neighbors, algorithm='kd_tree')

            # Train classifier
            start_time = datetime.datetime.now()
            
            #classifier.fit(X_train, y_train.values.ravel())
            classifier.fit(X_train, y_train.ravel())

            end_time = datetime.datetime.now()
            training_time_sec = (end_time - start_time).total_seconds()

            # Predict test set on trained classifier
            start_time = datetime.datetime.now()
            y_test_predicted = classifier.predict(X_test)
            end_time = datetime.datetime.now()
            testing_time_sec = (end_time - start_time).total_seconds()

            # Compute metrics
            acc = metrics.accuracy_score(y_test, y_test_predicted)
            f1 = f1_score(y_true=y_test, y_pred=y_test_predicted, average='weighted')

            # Store results
            result = type('',(object,),{'guid': str(uuid.uuid4()), 'classifier': classifier, 'algorithm': 'k-NN', 'n_neigbors': n_neighbors, 'training_time_sec': training_time_sec, 'testing_time_sec': testing_time_sec, 'acc': acc, 'f1': f1, 'y_test_predicted': y_test_predicted})()
            results.append(result)

            # Cache best result for confusion matrix return value
            if(bestResult is None or bestResult.f1 < result.f1):
                bestResult = result

    # Print results
    print(dataSetName)
    print('GUID | Algorithm | acc | f1 | training_time_sec | testing_time_sec')
    for res in results:
        print(res.guid + ' | k-NN (' + str(res.n_neigbors) + '-NN) | ' + str(round(res.acc, 3)) + ' | ' + str(round(res.f1, 3)) + ' | ' + str(res.training_time_sec) + ' sec | ' + str(res.testing_time_sec) + ' sec')
    print()

    return bestResult


In [21]:
# decision tree
def decision_tree(dataSetName, X_train, X_test, y_train, y_test, max_features_values):

    results = []
    bestResult = None

    for max_features in max_features_values:
            classifier = DecisionTreeClassifier(max_features=max_features, random_state=random_state) 

            # Train classifier
            start_time = datetime.datetime.now()
            classifier.fit(X_train, y_train.ravel())
            end_time = datetime.datetime.now()
            training_time_sec = (end_time - start_time).total_seconds()

            # Predict test set on trained classifier
            start_time = datetime.datetime.now()
            y_test_predicted = classifier.predict(X_test)
            end_time = datetime.datetime.now()
            testing_time_sec = (end_time - start_time).total_seconds()

            # Compute metrics
            acc = metrics.accuracy_score(y_test, y_test_predicted)
            f1 = f1_score(y_true=y_test, y_pred=y_test_predicted, average='weighted')

            # Store results
            result = type('',(object,),{'guid': str(uuid.uuid4()), 'classifier': classifier, 'algorithm': 'decision_tree', 'max_features': max_features, 'training_time_sec': training_time_sec, 'testing_time_sec': testing_time_sec, 'acc': acc, 'f1': f1, 'y_test_predicted': y_test_predicted})()
            results.append(result)

            # Cache best result for confusion matrix return value
            if(bestResult is None or bestResult.f1 < result.f1):
                bestResult = result

    # Print results
    print(dataSetName)
    print('GUID | Algorithm | acc | f1 | training_time_sec | testing_time_sec')
    for res in results:
        print(res.guid + ' | Decision Tree (max features: ' + str(res.max_features) + ') | ' + str(round(res.acc, 3)) + ' | ' + str(round(res.f1, 3)) + ' | ' + str(res.training_time_sec) + ' sec | ' + str(res.testing_time_sec) + ' sec')
    print()

    return bestResult

In [22]:
# svm
def svm_svc(dataSetName, X_train, X_test, y_train, y_test):

    results = []
    bestResult = None
    classifier = make_pipeline(StandardScaler(), SVC(random_state=random_state))

    # Train classifier
    start_time = datetime.datetime.now()
    classifier.fit(X_train, y_train.ravel())
    end_time = datetime.datetime.now()
    training_time_sec = (end_time - start_time).total_seconds()

    # Predict test set on trained classifier
    start_time = datetime.datetime.now()
    y_test_predicted = classifier.predict(X_test)
    end_time = datetime.datetime.now()
    testing_time_sec = (end_time - start_time).total_seconds()

    # Compute metrics
    acc = metrics.accuracy_score(y_test, y_test_predicted)
    f1 = f1_score(y_true=y_test, y_pred=y_test_predicted, average='weighted')

    # Store results
    result = type('',(object,),{'guid': str(uuid.uuid4()), 'classifier': classifier, 'algorithm': 'svm', 'training_time_sec': training_time_sec, 'testing_time_sec': testing_time_sec, 'acc': acc, 'f1': f1, 'y_test_predicted': y_test_predicted})()
    results.append(result)

    # Cache best result for confusion matrix return value
    if(bestResult is None or bestResult.f1 < result.f1):
        bestResult = result

    # Print results
    print(dataSetName)
    print('GUID | Algorithm | acc | f1 | training_time_sec | testing_time_sec')
    for res in results:
        print(res.guid + ' | SVM | ' + str(round(res.acc, 3)) + ' | ' + str(round(res.f1, 3)) + ' | ' + str(res.training_time_sec) + ' sec | ' + str(res.testing_time_sec) + ' sec')
    print()

    return bestResult

In [23]:
# random forests
def random_forests(dataSetName, X_train, X_test, y_train, y_test, numbers_of_trees, max_features_values):

    results = []
    bestResult = None

    for number_of_trees in numbers_of_trees:
        for max_features in max_features_values:

            classifier = RandomForestClassifier(n_estimators=number_of_trees, max_features=max_features, random_state=random_state)

            # Train classifier
            start_time = datetime.datetime.now()
            classifier.fit(X_train, y_train.ravel())
            end_time = datetime.datetime.now()
            training_time_sec = (end_time - start_time).total_seconds()

            # Predict test set on trained classifier
            start_time = datetime.datetime.now()
            y_test_predicted = classifier.predict(X_test)
            end_time = datetime.datetime.now()
            testing_time_sec = (end_time - start_time).total_seconds()

            # Compute metrics
            acc = metrics.accuracy_score(y_test, y_test_predicted)
            f1 = f1_score(y_true=y_test, y_pred=y_test_predicted, average='weighted')

            # Store results
            result = type('',(object,),{'guid': str(uuid.uuid4()), 'classifier': classifier, 'algorithm': 'random_forests', 'number_of_trees': number_of_trees, 'max_features': max_features, 'training_time_sec': training_time_sec, 'testing_time_sec': testing_time_sec, 'acc': acc, 'f1': f1, 'y_test_predicted': y_test_predicted})()
            results.append(result)

            # Cache best result for confusion matrix return value
            if(bestResult is None or bestResult.f1 < result.f1):
                bestResult = result

    # Print results
    print(dataSetName)
    print('GUID | Algorithm | acc | f1 | training_time_sec | testing_time_sec')
    for res in results:
        print(res.guid + ' | Random Forests (num trees: ' + str(res.number_of_trees) + ', max features: ' + str(res.max_features) + ') | ' + str(round(res.acc, 3)) + ' | ' + str(round(res.f1, 3)) + ' | ' + str(res.training_time_sec) + ' sec | ' + str(res.testing_time_sec) + ' sec')
    print()

    return bestResult
    


## Classification Application

### Find best Classifier

In [24]:
def kNN_with_parameters(data_set_name, X_train, X_test, y_train, y_test):
    n_neighbors_values = [1, 2, 3]
    return kNN(data_set_name, X_train, X_test, y_train, y_test, n_neighbors_values)

def decision_tree_with_parameters(data_set_name, X_train, X_test, y_train, y_test):
    max_features_values = [None, 'sqrt', 'log2'] 
    return decision_tree(data_set_name, X_train, X_test, y_train, y_test, max_features_values)

def random_forests_with_parameters(data_set_name, X_train, X_test, y_train, y_test):
    numbers_of_trees = [10, 100]
    max_features_values = ['sqrt', 'log2'] 
    return random_forests(data_set_name, X_train, X_test, y_train, y_test, numbers_of_trees, max_features_values)

classifiers = [
                kNN_with_parameters,
                decision_tree_with_parameters,
                svm_svc,
                random_forests_with_parameters
              ]

bestResult = None

X, y = shuffle(train_data_set_X, train_data_set_y, random_state=random_state)

# Prepare a train/test set split: split 2/3 1/3 into training & test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_state)

for indexClassifier, classifier in enumerate(classifiers):
    # do the actual classification
    result = classifier(data_set_name, X_train, X_test, y_train, y_test)

    # Cache best result for confusion matrix return value
    if(bestResult is None or bestResult.f1 < result.f1):
        bestResult = result
        bestResult.y_test = y_test

print('GUID of best classifier: ' + bestResult.guid)

breastcancer
GUID | Algorithm | acc | f1 | training_time_sec | testing_time_sec
aa1a316a-3d39-47bd-91cc-bec656985fc9 | k-NN (1-NN) | 0.874 | 0.873 | 0.003563 sec | 0.006142 sec
ff864127-adad-449f-8899-b9ee98e3265b | k-NN (2-NN) | 0.895 | 0.893 | 0.001962 sec | 0.005001 sec
c74c88a8-6cc1-4bc0-a7ab-8c6f5bc20c14 | k-NN (3-NN) | 0.884 | 0.884 | 0.002721 sec | 0.004027 sec

breastcancer
GUID | Algorithm | acc | f1 | training_time_sec | testing_time_sec
02a469a5-f3f9-4df7-bee1-cd8ed11646d8 | Decision Tree (max features: None) | 0.937 | 0.937 | 0.003035 sec | 0.000999 sec
fe336606-b529-4572-8a6b-5e3218204e6c | Decision Tree (max features: sqrt) | 0.937 | 0.937 | 0.001767 sec | 0.001001 sec
72960fb9-1725-488c-bdde-fc2328b301d7 | Decision Tree (max features: log2) | 0.947 | 0.947 | 0.002006 sec | 0.000959 sec

breastcancer
GUID | Algorithm | acc | f1 | training_time_sec | testing_time_sec
376f5290-a886-4204-afdb-684bf79c765a | SVM | 0.968 | 0.968 | 0.004311 sec | 0.001687 sec

breastcancer
GUID

### Retrain best Classifier and create Prediction

In [29]:
# Retrain best performing model on the whole training set
bestResult.classifier.fit(X, y.ravel())

# Create prediction for the test set
test_predicted = bestResult.classifier.predict(test_data_set_X)
predicted_csv = list(zip(*[test_data_set.loc[:,'ID'], [str(x).lower() for x in test_predicted]]))

# Save prediction in file
csv_file_path = data_set_name + '_prediction_' + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '_utc.csv'
np.savetxt(csv_file_path, 
           predicted_csv,
           header='ID,class',
           comments='',
           delimiter =",", 
           fmt ='%s')

print('Saved prediction file ' + csv_file_path)

Saved prediction file breastcancer_prediction_2022-06-05_08-10-20.csv
